init jeu de donnees et affichage des informations, on notte une difference entre les colonnes Total_Proteins de chaque table

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# A compléter
df_h1 = pd.read_json('./liver.json')
df_h2 = pd.read_csv('./liver.csv')
print(df_h1.info())
print(df_h2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         291 non-null    object 
 1   Gender                      291 non-null    object 
 2   Total_Bilirubin             290 non-null    float64
 3   Direct_Bilirubin            287 non-null    float64
 4   Alkaline_Phosphotase        290 non-null    float64
 5   Alamine_Aminotransferase    236 non-null    float64
 6   Aspartate_Aminotransferase  291 non-null    int64  
 7   Total_Proteins              289 non-null    float64
 8   Albumin                     290 non-null    float64
 9   Albumin_and_Globulin_Ratio  227 non-null    float64
 10  Outcome                     291 non-null    int64  
dtypes: float64(7), int64(2), object(2)
memory usage: 25.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (

combination donnees

avant de combiner les tables on fait l'harmonisation de la colonne Total_Proteins

In [15]:
def check_protein_format(protein_str):
    return isinstance(protein_str, str) and protein_str.strip().endswith('mg/dL')

def extract_protein_value(protein_str):
    if pd.isna(protein_str):
        return pd.NA
    if check_protein_format(protein_str):
        try:
            return float(protein_str.strip().replace('mg/dL','').strip())
        except:
            return pd.NA
    try:
        return float(protein_str)
    except:
        return pd.NA

# Aplicar la función al dataframe (df_h2 ya debería existir)
df_h2['Total_Proteins'] = df_h2['Total_Proteins'].apply(extract_protein_value)
df_h2['Total_Proteins'].head()

0    7800.0
1    7100.0
2    7700.0
3    8300.0
4    8000.0
Name: Total_Proteins, dtype: object

afficher information

In [5]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         612 non-null    object 
 1   Gender                      612 non-null    object 
 2   Total_Bilirubin             607 non-null    float64
 3   Direct_Bilirubin            607 non-null    float64
 4   Alkaline_Phosphotase        607 non-null    float64
 5   Alamine_Aminotransferase    236 non-null    float64
 6   Aspartate_Aminotransferase  291 non-null    float64
 7   Total_Proteins              610 non-null    object 
 8   Albumin                     606 non-null    float64
 9   Albumin_and_Globulin_Ratio  489 non-null    float64
 10  Outcome                     291 non-null    float64
 11  ALT                         254 non-null    float64
 12  AST                         321 non-null    float64
 13  Result                      321 non

Harmonisation de la colonne Total_Proteins